In [1]:
import pandas as pd
import json, os

"""
多少站
多少桩
功率
采样率
"""

'\n多少站\n多少桩\n功率\n采样率\n'

In [2]:
countries = pd.read_csv("Data2025\\CountryList.csv", dtype=str)
countries["ISO"].fillna("NA", inplace=True)

In [4]:
result = []
for country in countries["ISO"].to_list():
    path = os.path.join("Data2025", country + "Charger")
    with open(path + ".json", 'r') as j:
        data = json.load(j)
    chargingNum = 0
    power = set()
    fastStation = 0
    fastCharging = 0
    for i in data:
        NoP = 0 if i["NumberOfPoints"] is None else i["NumberOfPoints"]
        Q = 0
        fast = False
        for j in i["Connections"]:
            quantity = j["Quantity"]
            powerKW = j["PowerKW"]
            if quantity is None:
                Q += 1
            else:
                Q += quantity
            power.add(powerKW)
            if powerKW is not None and powerKW >= 20:
                fastCharging += 1
                if not fast:
                    fastStation += 1
                    fast = True
        chargingNum += max(NoP, Q)
    d = {
        "Country": country,
        "StationNum": len(data),
        "StationNum in Web": countries.loc[countries["ISO"] == country, "Location Number"].tolist()[0],
        "ChargingNum": chargingNum,
        "ChargingNum in Web": countries.loc[countries["ISO"] == country, "Station Number"].tolist()[0],
        "Power": power,
        "Fast Station Num (Contain at least one fast charging)": fastStation,
        "Fast Charging Num (Power >= 20kW)": fastCharging,
    }
    result.append(d)
pd.DataFrame(result).to_csv("statistic.csv", encoding="utf-8")